In [3]:
import numpy as np
import kagglehub
import os
from PIL import Image
from IPython.display import display

# Download MNIST CSV dataset
path = kagglehub.dataset_download("oddrationale/mnist-in-csv")
print("Path to dataset files:", path)


Path to dataset files: C:\Users\ma\.cache\kagglehub\datasets\oddrationale\mnist-in-csv\versions\2


In [ ]:

# Identify CSV files
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {path}")

csv_path = os.path.join(path, csv_files[0])

# Load data
data = np.genfromtxt(csv_path, delimiter=',', skip_header=1)
labels = data[:, 0].astype(int)
pixels = data[:, 1:].astype(np.uint8)
scaled_pixels = pixels / 255.0


class Layer:
    def __init__(self, number, size, inputs=1):
        self.number = number
        self.size = size
        self.data = np.zeros((inputs, size), dtype=np.float32)

class Weight:
    def __init__(self, prev, next):
        self.value = np.random.randn(prev.size, next.size).astype(np.float32)

class Bias:
    def __init__(self, next):
        self.value = np.random.randn(1, next.size).astype(np.float32)

class Neural:
    def __init__(self, activation):
        self.activation = activation
        self.layers = []
        self.weights = []
        self.biases = []

    def add_layer(self, layer):
        self.layers.append(layer)
        if len(self.layers) > 1:
            self.weights.append(Weight(self.layers[len(self.layers)-2], self.layers[len(self.layers)-1]))
            self.biases.append(Bias(self.layers[len(self.layers)-1]))

    def forward(self):
        for i in range(1, len(self.layers)):
            prev_data = self.layers[i - 1].data     
            w = self.weights[i - 1].value            
            b = self.biases[i - 1].value            
            z = np.dot(prev_data, w) + b            
            self.layers[i].data = self.activation(z)

    @property
    def output(self):
        return self.layers[-1].data

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# cost function = (predicted-actual)^2 
def costFunction(predicted, actual):
    return np.sum((predicted - actual) ** 2)



In [15]:
input_taken = 5
indices = list(range(input_taken))

input_data = scaled_pixels[indices] 

actual = np.zeros((input_taken, 10), dtype=np.float32)
for i in range(len(indices)):
    idx = indices[i]
    label = labels[idx]
    actual[i, label] = 1
    display(Image.fromarray(pixels[idx].reshape(28,28), mode = 'L'))


print(actual)


[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [18]:

model = Neural(sigmoid)

input_layer = Layer(1, 784, inputs=input_taken)
input_layer.data = input_data

model.add_layer(input_layer)
model.add_layer(Layer(2, 107, inputs=input_taken))
model.add_layer(Layer(3, 26, inputs=input_taken))
model.add_layer(Layer(4, 10, inputs=input_taken))

model.forward()

result = np.argmax(model.output, axis=1)
true_labels = labels[indices]

print("Predicted Labels:", result)
print("True Labels     :", true_labels)
print(model.output)

cost = costFunction(model.output,actual)
print("Cost:", cost)


Predicted Labels: [4 4 4 4 4]
True Labels     : [7 2 1 0 4]
[[0.82280676 0.04305381 0.03015486 0.2442052  0.99749513 0.30531814
  0.07816139 0.96169761 0.72455942 0.33569808]
 [0.99559736 0.16306018 0.58997987 0.72785294 0.99878518 0.98395283
  0.36222928 0.61302063 0.92518356 0.96492216]
 [0.94067073 0.01741994 0.15857719 0.47176207 0.99874375 0.88241846
  0.33743436 0.89672073 0.86643822 0.33289913]
 [0.97707188 0.03860646 0.48321408 0.23905905 0.98904502 0.98512746
  0.29384941 0.74176616 0.49925619 0.70790269]
 [0.99161329 0.26063154 0.05096999 0.30698229 0.99863174 0.99683426
  0.85172075 0.98767413 0.72415343 0.2579037 ]]
Cost: 22.16391710850425
